# Old Cyclus Tutorial Notes

## Original example scenario:
-  1 natural uranium mine
-  1 enrichment facility producing 4% enriched fuel
-  1 LWR consuming fresh fuel and producing SNF <font color='purple'>
    -  Does the scenario assume it starts at year 1?
    -  How hard would it be to incorporate a mix of MOX and UOX fuel?
    -  How would it work if someone wanted to start a model simulating a scenario with an older reactor?
        -  Or, do you just always start at year 1, and change simulator duration?</font>
-  1 repository for SNF
    
## Notes: Time Steps:
Time step method used - at each time step, agents (the enrichment facility, LWR, etc.) can take certain actions:

-  Deployment: an agent is added
-  Each agent prepares to exchange materials
-  Agents exchange materials
-  Each agent performs its post-exchange action
-  Decommissioning: an agent leaves the system

In order to correctly implement this, the user must give:

-  Duration: The number of months to be simulated
-  Start Month: the first month of the simulation
-  Start Year:  the first year of the simulation <font color='purple'>
-  Decay Treatment:  Turn off all decay ("never") or let the archetypes handle it
-  Simulation Handle: An optional unique identifier for this particular simulation
-  Description: A brief description for your benefit.
    -  Are these options (or the "never" decay tag) unique to Cyclist?
</font>

## Notes: Archetypes:
An <font color='blue'>archetype</font> is a model of a particular facility, such as an LWR or enrichment plant.  using archetypes, the user can switch between different "versions" of a particular facility, changing how it interacts with other facilities, or changing the physics of the facility, or both.

-  Archetypes can also be chosen based on their complexity.  As expected, a more detailed/complex model will require more computing resources than a simplified model.  The user should consider the fidelity and performance of their simulation when choosing archetypes.

There are some basic Cycamore archetypes, which are good for simple simulations, or for modeling portions of the simulation that are necessary, but are not the focus of the analysis.

-  Source: any facility that produces fresh material.  Mines, enrichment facilities, fabrication facilities, etc.
-  Enrichment: for enriching U235, as seen above, could be a "source" archetype.  Has a constrained total enrichment capacity.
    -  <font color='purple'>What is this limit?</font>
-  Reactor: uses simple fuel recipes, loads batches at regular intervals
    -  <font color='purple'>Does this assume all fuel is new each time it's reloaded?  Is there a way to model core configurations?</font>
-  Fabrication: mixes fissile/fissionable material to approximate a fuel recipe, using the d-factor approach.
     -  <font color='purple'>What is the d-factor approach?</font>
-  Sink: Any facility that permanently holds SNF.  Geological repositories, interim storage facilities, etc.

<font color='purple'>The portion of the old tutorial about adding archetypes seems pretty unique to Cyclist, and in Gyutae's examples, I can't see anything corresponding to adding in an already made archetype.  I can see that he calculates fresh and spent fuel recipes, and pulls things like deployment data from PRIS.  He imports the jinja2 library - is this template maker made for Cyclus, does it use archetypes?</font>

## Configuring Archetypes:
Generally, archetypes will allow for certain parameters to be designated by the user.  For example, the user might adjust the power output of their reactor.  An archetype that has been modified from its default settings is referred to as a "<font color='blue'>prototype</font>".
<br><br>
<font color='purple'>Cyclist uses a GUI for making adjustments to archetypes, how is this done outside of Cyclist?</font>

## Notes: Commodities
Cyclus simulates exchange between facilities using a mechanism called the Dynamic Resource Exchange (DRE).  The commodity indicates which facilities will be interested in trading with each other through DRE.  So, a commodity can be thought of as a unique name for the producers and consumers of a certain resource.  This won't have its own composition - that will be determined by the facilities during the simulation.
<br><br>
<font color='purple'>How would you add commodities outside the Cyclist GUI?</font>

## Notes: Material Recipes
Archetypes may want specified recipes for input/output.  A <font color='blue'>recipe</font> is the isotopic composition, either in mass or atom fraction.
<br><br>
<font color='purple'>In the Cyclist Tutorial, it explicitly gives the fuel recipes for the user to put in.  I can see in Gyutae's notebook that he has functions for creating fuel recipes, and he uses these recipes later.  For our tutorial, we should be able to skip the step of generating fuel recipes using decay as Gyutae did, and instead, directly have the user declare the recipe arrays.  The jinja2 template rendering portion takes the fuel recipes in the format of a dictionary object with {isotope: mass fraction composition} as {key : value} pair, notably.
<br><br>
But it looks like after getting a recipe file rendered, another function still had to be used to render the Cycamore reactor specs (in Gyutae's, this function is called `write_reactors`).  This creates .xml files.  This is not the final step, however - Gyutae renders deployment data as .xml, then defines a function `render_cyclus`, which creates the finalized Cyclus input.  This .xml file can be run directly using a command in the terminal.  The rendered Cyclus input references the other .xml files that were made in previous steps, which is why the previous steps of getting recipe and deployment data were done the way they were.  Gyutae gives a brief explanation of implementing .xml files as he did.</font>

## Notes: Regions & Institutions
Agents in Cyclus exist in a hierarchy:  Region agent > Institution agent > Facility agent.  This allows the simulation to factor geological concerns into modeling interactions between agents.  For example, during trading commodities.  Every simulation needs at least one region and one institution.  The default options are "Null", e.g. NullInst and NullRegion.  NullRegion holds the set of institutions, and NullInst holds the set of facilities that are deployed at the start of the simulation.
<br><br>
<font color='purple'>Cyclist has you manually set the inital facilities that are deployed - how is this done with python?</font>

## How to run Cyclus
<font color='purple'>The tutorial tells the user how to run Cyclus from a remote source.  Since we were to assume the user would have the relevant software already installed, I'd assumed we'd only cover command line commands.  Should our tutorial cover both?
<br><br>
Related to this, should we give a brief blurb about sqlite database files, or is that something they should learn on their own? (It's true that SQL is not unique to Cyclus, but if we want to be new user friendly, it might be helpful (I, at least, had not encountered SQL before this project)).</font>

## Data Exploration
<font color='purple'>The Cyclus tutorial uses Cyclist to examine the output within its GUI.  It lets users look at multiple simulations at once.  It also has tabs for specific output tables and for the fields within each table.  For the tutorial we make, analysis will be done through the jupyter notebook.  The tutorial doesn't handle running more than one scenario at once, and so ours shouldn't need to either.  The original tutorial has the user make plots for the inventory over time, but I think that's less because it's an essential value, and more because it automatically appears in the Cyclist table.
<br><br>
It might be helpful to build the simulation, and then choose what data analysis would be helpful based on that.  Gyutae's plots include things like # reactors over time or fuel utilization - I think simple concepts like this, which anyone should know the meaning of, would be good to pick.  (I'm thinking of the scenario where we get a freshman or some such that maybe hasn't taken many NPRE classes before - I'd like to try and pick things they could infer the meaning of).</font>

## Advanced Data Exploration:
<font color='purple'>This portion introduces something called a flow view, which is unique to Cyclus.  It shows the amount of material flowing between facilities.  I can see how this is implemented in Cyclist, I'm not sure if this could be replicated using jupyter notebooks?  It looks like it's plotting the amount of material that facility X sends to facility Y at each time step - for example, the amount a reactor sends to the repository at each time step.  Depending on the data Cyclus outputs, we might be able to create the same sort of flow plots outside Cyclus.
<br><br> 
This portion also details how you can access the tables that aren't automatically included in the table tab when Cyclist opens, but if we're pulling directly into python from an sqlite database, that's not a problem for us, once we learn how to do it.  (as a side note, it looks like Gyutae used a function from an sqlite library to create a "cursor" to the sqlite database in question, and then called that cursor like you would any other imported library.  Specifically, `cursor.execute(SQLlite subscripting)` to get the tables he wants.  Software Carpentry has a lesson on SQLite, the subscripting isn't too bad.) </font>

## Exercise 2:
The old tutorial has a second set of exercises with a fewer explicit directions.  It:
<br><br>
-  Has the user add recycling into the simulation, and model a fuel cycle with recycling and MOX fuel.
    -  It goes through the same general steps, but adds in separation facilities, the MOX fuel recipe, etc.